### Q1




In [2]:
import random

piece_values = {
    'P': 1, 'N': 3, 'B': 3, 'R': 5, 'Q': 9, 'K': 1000,
    'p': -1, 'n': -3, 'b': -3, 'r': -5, 'q': -9, 'k': -1000, '.': 0
}


board = [
    ['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r'],
    ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
    ['.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.', '.'],
    ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
    ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']
]


def evaluate_board(board):
    return sum(piece_values[board[row][col]] for row in range(8) for col in range(8))


def get_legal_moves(board, color):
    moves = []
    for row in range(8):
        for col in range(8):
            piece = board[row][col]
            if (color == 'white' and piece.isupper()) or (color == 'black' and piece.islower()):
                moves.extend(generate_piece_moves(board, row, col, piece))
    return moves



def generate_piece_moves(board, row, col, piece):
    moves = []
    directions = {
        'P': [(-1, 0)], 'p': [(1, 0)],
        'N': [(2, 1), (2, -1), (-2, 1), (-2, -1),
              (1, 2), (-1, 2), (1, -2), (-1, -2)],
        'B': [(1, 1), (1, -1), (-1, 1), (-1, -1)],
        'R': [(1, 0), (-1, 0), (0, 1), (0, -1)],
        'Q': [(1, 1), (1, -1), (-1, 1), (-1, -1),
              (1, 0), (-1, 0), (0, 1), (0, -1)],
        'K': [(1, 1), (1, -1), (-1, 1), (-1, -1),
              (1, 0), (-1, 0), (0, 1), (0, -1)]
    }


    sliding_pieces = ['B', 'R', 'Q']
    piece_upper = piece.upper()

    if piece_upper in directions:
        for dr, dc in directions[piece_upper]:
            for step in range(1, 8):
                new_r, new_c = row + dr * step, col + dc * step
                if not (0 <= new_r < 8 and 0 <= new_c < 8):
                    break
                dest = board[new_r][new_c]

                if dest == '.':
                    moves.append(((row, col), (new_r, new_c)))
                elif (dest.islower() if piece.isupper() else dest.isupper()):
                    moves.append(((row, col), (new_r, new_c)))
                    break
                else:
                    break
                if piece_upper not in sliding_pieces:
                    break
    return moves



def apply_move(board, move):
    new_board = [row[:] for row in board]
    (r1, c1), (r2, c2) = move
    new_board[r2][c2] = new_board[r1][c1]
    new_board[r1][c1] = '.'
    return new_board


def beam_search(board, color='white', beam_width=3, depth=3):
    beams = [(board, [])]

    for _ in range(depth):
        new_beams = []
        for current_board, move_seq in beams:
            legal_moves = get_legal_moves(current_board, color)

            scored_moves = []
            for move in legal_moves:
                new_board = apply_move(current_board, move)
                score = evaluate_board(new_board)
                scored_moves.append((score, move, new_board))

            scored_moves.sort(reverse=(color == 'white'), key=lambda x: x[0])
            top_moves = scored_moves[:beam_width]

            for score, move, new_board in top_moves:
                new_beams.append((new_board, move_seq + [move]))

        beams = new_beams
        color = 'black' if color == 'white' else 'white'

    best_board, best_moves = max(beams, key=lambda x: evaluate_board(x[0]))
    best_score = evaluate_board(best_board)
    return best_moves, best_score


def move_to_str(move):
    (r1, c1), (r2, c2) = move
    return f"{chr(c1 + ord('a'))}{8 - r1} -> {chr(c2 + ord('a'))}{8 - r2}"


best_moves, best_score = beam_search(board, color='white', beam_width=3, depth=3)


print("Best Move Sequence:")
for move in best_moves:
    print(move_to_str(move))
print("Evaluation Score:", best_score)


Best Move Sequence:
a2 -> a3
b8 -> c6
a3 -> a4
Evaluation Score: 0


### Q2

In [ ]:
import random
import math

def distance(point1, point2):
    return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

def total_route_distance(route):
    return sum(distance(route[i], route[i + 1]) for i in range(len(route) - 1)) + distance(route[-1], route[0])

def hill_climbing(locations, max_iterations=1000):
    current_route = locations[:]
    random.shuffle(current_route)
    current_distance = total_route_distance(current_route)

    for _ in range(max_iterations):
        new_route = current_route[:]
        i, j = random.sample(range(len(locations)), 2)
        new_route[i], new_route[j] = new_route[j], new_route[i]
        new_distance = total_route_distance(new_route)

        if new_distance < current_distance:
            current_route, current_distance = new_route, new_distance

    return current_route, current_distance


delivery_points = [(0, 0), (2, 3), (5, 4), (7, 1), (6, 6), (3, 7)]
optimized_route, min_distance = hill_climbing(delivery_points)

print("Optimized Route:", optimized_route)
print("Total Distance:", min_distance)


Optimized Route: [(6, 6), (5, 4), (7, 1), (0, 0), (2, 3), (3, 7)]
Total Distance: 23.803621626079284


### Q3


In [4]:
import random
import math

CITIES = ['Kalinin', 'Stalingrad', 'Moscow', 'Kiev', 'Mogilev',
          'Dnieperpetrovsk', 'Leningrad', 'Rostov', 'Baku', 'Vladivostok']

CITIES_COORDS = {city: (random.randint(0, 100), random.randint(0, 100)) for city in CITIES}

POPULATION_SIZE = 100
GENERATIONS = 1000
TOURNAMENT_SIZE = 5
MUTATION_RATE = 0.3

def euclidean_dist(city1, city2):
    x1, y1 = CITIES_COORDS[city1]
    x2, y2 = CITIES_COORDS[city2]
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def generate_individual():
    return random.sample(CITIES, len(CITIES))

def fitness(individual):
    total = sum(euclidean_dist(individual[i], individual[i + 1]) for i in range(len(individual) - 1))
    total += euclidean_dist(individual[-1], individual[0])  # return to start
    return 1 / total

# ordered crossover
def total_distance(individual):
    return sum(euclidean_dist(individual[i], individual[i + 1]) for i in range(len(individual) - 1)) + \
           euclidean_dist(individual[-1], individual[0])

def tournament_selection(population):
    tournament = random.sample(population, TOURNAMENT_SIZE)
    return max(tournament, key=fitness)

def ordered_crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None] * size
    child[start:end] = parent1[start:end]
    remaining = [city for city in parent2 if city not in child]
    child = [remaining.pop(0) if gene is None else gene for gene in child]
    return child

def swap_mutation(individual, rate=MUTATION_RATE):
    if random.random() < rate:
        i, j = random.sample(range(len(individual)), 2)
        individual[i], individual[j] = individual[j], individual[i]
    return individual

def genetic_algorithm_TSP():
    population = [generate_individual() for _ in range(POPULATION_SIZE)]

    for generation in range(GENERATIONS):
        new_population = []
        for _ in range(POPULATION_SIZE // 2):
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            child1 = swap_mutation(ordered_crossover(parent1, parent2))
            child2 = swap_mutation(ordered_crossover(parent2, parent1))
            new_population.extend([child1, child2])
        population = new_population

        if generation % 10 == 0 or generation == GENERATIONS - 1:
            best = max(population, key=fitness)
            print(f"Generation {generation}: Best Distance = {total_distance(best):.2f}")

    best_individual = max(population, key=fitness)
    return best_individual, fitness(best_individual), total_distance(best_individual)

best_route, best_fit, best_dist = genetic_algorithm_TSP()

print("\nBest Route Found:")
for i, city in enumerate(best_route):
    print(f"{i+1}. {city} at {CITIES_COORDS[city]}")
print(f"\nTotal Distance: {best_dist:.2f}")
print(f"Fitness Score: {best_fit:.6f}")


Generation 0: Best Distance = 426.44
Generation 10: Best Distance = 342.49
Generation 20: Best Distance = 342.49
Generation 30: Best Distance = 342.49
Generation 40: Best Distance = 342.49
Generation 50: Best Distance = 342.49
Generation 60: Best Distance = 342.49
Generation 70: Best Distance = 342.49
Generation 80: Best Distance = 342.49
Generation 90: Best Distance = 342.49
Generation 100: Best Distance = 342.49
Generation 110: Best Distance = 342.49
Generation 120: Best Distance = 342.49
Generation 130: Best Distance = 342.49
Generation 140: Best Distance = 333.56
Generation 150: Best Distance = 333.56
Generation 160: Best Distance = 333.56
Generation 170: Best Distance = 333.56
Generation 180: Best Distance = 333.56
Generation 190: Best Distance = 333.56
Generation 200: Best Distance = 333.56
Generation 210: Best Distance = 333.56
Generation 220: Best Distance = 333.56
Generation 230: Best Distance = 333.56
Generation 240: Best Distance = 333.56
Generation 250: Best Distance = 333.